In [1]:
from blockworld import Blockworld
from simple_spatial import SimpleSpatial
from agent import _considering_a_subset_of_goals, _subset_of_ordered_goals, bfs_search, _all_incomplete_goal_combinations

# Blockworld

In [2]:
problem = Blockworld(
    (('A', 'C'), ('B',), ()),
    (
        Blockworld.make_above_predicate('B', 'C'),
        Blockworld.make_above_predicate('A', 'B'),
    ),
)

print('Considering a subset of goals (of size k)')
print("\nSussman's anomaly!")
print(problem.render(problem.initial))
print("\nwon't halt when k=1.")
_considering_a_subset_of_goals(problem)
print('\nbut it does when k=2')
_considering_a_subset_of_goals(problem, k=2)

Considering a subset of goals (of size k)

Sussman's anomaly!
...
C..
AB.


won't halt when k=1.
Did not find solution in 19 actions

but it does when k=2
Found solution in 3 actions [('C', 2), ('B', 2), ('A', 2)]


In [3]:
print(
    '\nMaking things more complex, and demonstrating a case where a subset (of size k=1) of goals '
    'may be suboptimal because of intermediate states.')
# This problem is degenerate when you consider all goal subsets.
# By assuming goals are ordered, we avoid some of those issues.
problem = Blockworld(
    (('D', 'A'), ('C', 'B'), ()),
    [
        Blockworld.make_above_predicate(top, bottom)
        for (top, bottom) in [('C', 'D'), ('B', 'C'), ('A', 'B')]
    ],
)
print(problem.render(problem.initial))
_considering_a_subset_of_goals(problem, k=1, subset_fn=_subset_of_ordered_goals)

print('By swapping the order of our start state columns, this can take longer or shorter:')
problem.initial = (('C', 'B'), ('D', 'A'), ())
_considering_a_subset_of_goals(problem, k=1, subset_fn=_subset_of_ordered_goals)


Making things more complex, and demonstrating a case where a subset (of size k=1) of goals may be suboptimal because of intermediate states.
...
...
AB.
DC.

Found solution in 5 actions [('A', 2), ('B', 2), ('C', 0), ('B', 0), ('A', 0)]
By swapping the order of our start state columns, this can take longer or shorter:
Found solution in 6 actions [('B', 2), ('A', 2), ('C', 1), ('A', 0), ('B', 1), ('A', 1)]


### Considering all subsets of goals

I've written up two functions to consider all subsets of goals (one considers all goals, the other only incomplete goals). The algorithm performs breadth-first search using each subset and picks the action plan that is the shortest. There are many problems with this:
  - It's clear that we want to avoid considering complete goals as complete goals don't require moves to be satisfied.
  - However, it may be important to consider currently satisfied goals that might need to temporarily not satisfy a goal. Example below...

So, this entire enterprise of considering all subsets might be a bad idea, and I can't think of a good alternative besides the "subsets of ordered goals" I propose elsewhere.

In [4]:
problem = Blockworld(
    (('D', 'A'), ('C', 'B'), ()),
    [
        Blockworld.make_above_predicate(top, bottom)
        for (top, bottom) in [('C', 'D'), ('B', 'C'), ('A', 'B')]
    ],
)
print('Optimal path is {} steps'.format(len(bfs_search(problem))))
_considering_a_subset_of_goals(problem, k=2)

Optimal path is 5 steps
Did not find solution in 19 actions


In [5]:
# Can't even find optimal solution using all incomplete goal combinations because some goals start off satisfied...
_considering_a_subset_of_goals(problem, k=3, subset_fn=_all_incomplete_goal_combinations, debug=True)

State at t=1
...
...
AB.
DC.

State at t=2
...
...
A..
DCB

State at t=3
...
...
..A
DCB

State at t=4
...
...
C.A
D.B

State at t=5
...
...
C..
DAB

State at t=6
...
B..
C..
DA.

State at t=7
A..
B..
C..
D..

Found solution in 6 actions [('B', 2), ('A', 2), ('C', 0), ('A', 1), ('B', 0), ('A', 0)]


In [6]:
print('\nThis is an example from my xls writeup that is interesting for k=2')
problem = Blockworld(
    (('E', 'A'), ('D', 'C', 'B'), ()),
    [
        Blockworld.make_above_predicate(top, bottom)
        for (top, bottom) in [('D', 'E'), ('C', 'D'), ('B', 'C'), ('A', 'B')]
    ],
)
print(problem.render(problem.initial))

print('Optimal solution has {} steps.'.format(len(bfs_search(problem))))

print('When k=2, we can only sometimes solve optimally:')
_considering_a_subset_of_goals(problem, k=2, subset_fn=_subset_of_ordered_goals)


This is an example from my xls writeup that is interesting for k=2
...
...
.B.
AC.
ED.

Optimal solution has 7 steps.
When k=2, we can only sometimes solve optimally:
Found solution in 7 actions [('A', 2), ('B', 2), ('C', 2), ('D', 0), ('C', 0), ('B', 0), ('A', 0)]


In [7]:
print('But we sometimes can\'t depending on problem state:')
problem.initial = (('D', 'C', 'B'), ('E', 'A'), ())
_considering_a_subset_of_goals(problem, k=2, subset_fn=_subset_of_ordered_goals)

But we sometimes can't depending on problem state:
Found solution in 8 actions [('B', 2), ('A', 2), ('C', 2), ('D', 1), ('C', 1), ('A', 0), ('B', 1), ('A', 1)]


# Simple spatial domain

In [8]:
m = [
    ".....",
    "....B",
    ".....",
    "A..A.",
    ".S...",
]
problem = SimpleSpatial(((1, 4), ()), m, [
    SimpleSpatial.make_visited_goal(symbol) for symbol in ['A', 'B']])
print(problem.render(problem.initial))

optimal = bfs_search(problem)
print('Optimal ({} steps): {}'.format(len(optimal), optimal))

print('Considering one goal:')
_considering_a_subset_of_goals(problem, k=1, subset_fn=_subset_of_ordered_goals)

.....
....B
.....
A..A.
.@...

Optimal (6 steps): [(1, 0), (1, 0), (0, -1), (1, 0), (0, -1), (0, -1)]
Considering one goal:
Found solution in 8 actions [(-1, 0), (0, -1), (1, 0), (1, 0), (1, 0), (1, 0), (0, -1), (0, -1)]


Since we're considering subsets of our goals and assuming they're ordered in optimal/required completion ordering, it's important to write our goals in the right order. Here's an example where the goals aren't correctly ordered when specifying the program, which can cause suboptimal paths:

In [9]:
problem = SimpleSpatial(((1, 4), ()), m, [
    SimpleSpatial.make_visited_goal(symbol) for symbol in ['B', 'A']])

print('Considering one goal at a time, out of order:')
_considering_a_subset_of_goals(problem, k=1, subset_fn=_subset_of_ordered_goals)

Considering one goal at a time, out of order:
Found solution in 9 actions [(1, 0), (1, 0), (1, 0), (0, -1), (0, -1), (0, -1), (-1, 0), (0, 1), (0, 1)]


In [10]:
print('\nMore complex example considering 3 goals.')
m = [
    "...C.",
    "B..B.",
    ".....",
    "A..A.",
    ".S...",
]
problem = SimpleSpatial(((1, 4), ()), m, [
    SimpleSpatial.make_visited_goal(symbol) for symbol in ['A', 'B', 'C']])
print(problem.render(problem.initial))

optimal = bfs_search(problem)
print('Optimal ({} steps): {}'.format(len(optimal), optimal))

print('Considering 2 goals:')
_considering_a_subset_of_goals(problem, k=2, subset_fn=_subset_of_ordered_goals)

print('Considering 1 goal:')
_considering_a_subset_of_goals(problem, k=1, subset_fn=_subset_of_ordered_goals)



More complex example considering 3 goals.
...C.
B..B.
.....
A..A.
.@...

Optimal (6 steps): [(1, 0), (1, 0), (0, -1), (0, -1), (0, -1), (0, -1)]
Considering 2 goals:
Found solution in 8 actions [(-1, 0), (0, -1), (1, 0), (1, 0), (1, 0), (0, -1), (0, -1), (0, -1)]
Considering 1 goal:
Found solution in 8 actions [(-1, 0), (0, -1), (0, -1), (0, -1), (1, 0), (1, 0), (1, 0), (0, -1)]
